In [ ]:
pip install tqdm

In [ ]:
pip install datasets

In [ ]:
import torch
from tqdm import tqdm
import pandas as pd
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset



device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype
)

# model = model.to_bettertransformer()
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
sample = dataset[0]["audio"]

# 음성에서 텍스트로 변환하는 함수
def convert_audio_to_text(audio_path, pipeline):
    result = pipeline(audio_path)
    return result["text"]

# 결과를 저장할 리스트
results = []

# 음성을 텍스트로 변환하고 결과를 리스트에 추가
for i in tqdm(range(1, 2063), desc="Processing"):
    audio_path = f"/content/drive/MyDrive/sesac/fail_audio_22/fail{i}_audio.wav"
    text_result = convert_audio_to_text(audio_path, pipe)
    results.append(text_result)

# 결과를 DataFrame으로 변환
df = pd.DataFrame({"Text": results})

# CSV 파일로 저장
df.to_csv("fail_results_22_231.csv", index=False)

# 결과 출력
df